# Domaca naloga 1.2

Avtomatizirana izbira najbosega modela in nastavitev hiper parametrov.


In [43]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, rand, fmin, Trials, space_eval
from hyperopt import pyll, base

import warnings

# ne izpisuje opozoril
warnings.simplefilter(action='ignore', category=FutureWarning)


data = pd.read_csv('podatki.csv')
X = data.drop('y', axis=1)
Y = data['y']
clas = Y.unique()
Y = Y.replace(clas[0], 0)
Y = Y.replace(clas[1], 1)

#print(Y)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc,columns=X.columns)
X_test_sc = scaler.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc,columns=X.columns)

print(X_train_sc)

           x1        x2        x3        x4        x5        x6        x7  \
0   -0.448816 -0.564758 -0.537651 -0.538668 -0.853073 -0.118410 -0.296610   
1   -0.448816 -1.159262  2.267118  0.382119  1.375031  0.427472  0.155352   
2   -0.448816  0.703517 -0.537651 -0.538668 -0.853073 -0.245620 -0.597918   
3   -0.448816 -0.445857 -0.537651 -0.334049 -0.853073 -0.046392  0.004698   
4   -0.448816  2.685197 -0.537651 -0.743288 -0.853073 -0.221292 -0.899227   
..        ...       ...       ...       ...       ...       ...       ...   
957  0.778359 -0.604392 -0.329890 -0.231739  0.652085 -0.114657 -0.145956   
958 -0.244287  0.029746 -0.329890 -0.334049  1.969098 -0.195753 -0.899227   
959  0.982888 -0.445857  0.605033  1.405217  1.071178  0.085540  0.607315   
960 -0.448816 -0.089155 -0.537651 -0.743288 -0.853073 -0.175297  0.155352   
961  0.164771 -0.961094  0.501153  0.382119  0.945591 -0.048934  0.004698   

           x8        x9       x10  ...       x21       x22       x23  \
0  

Prvi poskus se je izkazal da zadeva predolgo melje zato sem se odlocil za mansanje dimenzij podatkov. Izbral sem pristop **feature_importances_** tako da sem izbral prvih 7 najpomebnejsih znacilk, ker je pri tej znacilki AUC presegel vrednost 0.8.

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate


np.random.seed(0)

rfo = RandomForestClassifier().fit(X_train, y_train)

col_name = X.columns
imprf = rfo.feature_importances_


sortind = np.argsort(-imprf)
sort_imprf = imprf[sortind]
sort_name = col_name[sortind]
#print(X_train[sort_name[0:0]])

for i in range(1, len(sort_name)):
    x_train = X_train[sort_name[0:i]]
    x_test = X_test[sort_name[0:i]]
    rf_imp = RandomForestClassifier()
    cv = cross_validate(rf_imp, x_train, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)
    rf_imp.fit(x_train, y_train)
    pred = rf_imp.predict(x_test)
    print(f"cv = {np.mean(cv['test_score'])}")
    print(f"test = {roc_auc_score(pred, y_test)}")
    if roc_auc_score(pred, y_test) > 0.8:
        print(i)
        break
    
print(sort_name[0:i])


cv = 0.863463708557602
test = 0.7438640654499685
cv = 0.8569429683217973
test = 0.6790650406504065
cv = 0.890874744172675
test = 0.702527646129542
cv = 0.9126591789581873
test = 0.7174418604651163
cv = 0.9186002122358963
test = 0.7957535885167464
cv = 0.9173449704734751
test = 0.7499074074074075
cv = 0.9276619113121081
test = 0.8087182823682498
7
Index(['x1', 'x5', 'x3', 'x2', 'x4', 'x21', 'x27'], dtype='object')


In [49]:
print(sortind[0:i])
#print(X_test_sc[:,sortind[0:i]])
print(X_test_sc[sort_name[0:i]])

[ 0  4  2  1  3 20 26]
           x1        x5        x3        x2        x4       x21       x27
0   -0.244287 -0.068979 -0.329890 -0.525125 -0.334049 -0.213625  0.044978
1   -0.448816 -0.853073 -0.537651 -0.287323 -0.640978 -0.688237 -0.528123
2   -0.448816  0.558013  0.085631 -1.000728 -0.538668 -0.292727 -0.528123
3   -0.244287  0.685951  1.228315 -0.247689  0.586739  0.656497  1.764283
4   -0.039758  0.424430  1.539956 -0.644025  1.712146  1.566170  1.764283
..        ...       ...       ...       ...       ...       ...       ...
236  8.754996  1.462048  1.332195 -0.723293  2.530624  3.504168  3.340312
237  0.164771 -0.180925 -0.537651  0.426082 -0.538668 -0.015870 -0.528123
238  2.414592  0.640326  2.163238 -0.604392 -0.027120  3.108658 -0.528123
239 -0.448816 -0.853073 -0.537651 -0.723293 -0.538668 -0.569584 -0.528123
240 -0.448816 -0.853073 -0.537651 -0.287323 -0.436359 -0.530033 -0.241572

[241 rows x 7 columns]


In [50]:
gnezdeni_prostor = {
    "algo": hp.choice('algo', [
        {
            'name': 'rf',
            'max_depth': hp.choice('max_depth', list(range(2, 50, 5))),
            'min_samples_split' : hp.choice('min_samples_split', list(range(1, 20, 2))),
            'min_samples_leaf' : hp.choice('min_samples_leaf', [1, 2, 3])

        },
        {
            'name': 'knn',
            'n_neighbors' : hp.choice('n_neighbors', list(range(2, 30, 2)))
        },
        {
            'name': 'svm',
            'C': hp.choice('C', [1.0, 10.0]),
            'kernel': hp.choice('kernel', [
                {
                    'type': 'linear'  # linearno
                },
                {
                    'type': 'sigmoid'
                },
                {
                    'type': 'poly'
                }
            ]),
        },
    ])
}


In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate


i = 7
#Xi_train = X_train[sort_name[0:i]]
#Xi_test =  X_test[sort_name[0:i]]

Xi_train_sc = X_train_sc[sort_name[0:i]]
Xi_test_sc =  X_test_sc[sort_name[0:i]]

def kriteri_fn(params):
    a = params["algo"]
    algo_name = a["name"]
    if algo_name == "knn":
        model = KNeighborsClassifier(n_neighbors=a["n_neighbors"])

    elif algo_name == "rf":
        model = RandomForestClassifier(max_depth=a['max_depth'], 
                                       min_samples_split=a['min_samples_split'], 
                                       min_samples_leaf=a['min_samples_leaf'])

    elif algo_name == "svm":
        kernel = a["kernel"]["type"]
        C = a['C']
        model = SVC(kernel=kernel, C=C)
    else:
        raise ValueError("Wrong setings!")
    
    cv = cross_validate(model, Xi_train_sc, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)

    #model.fit(X_train, y_train)
    #pred = model.predict(X_test)
    #return 1 - roc_auc_score(y_test, pred) 
    return 1 - np.mean(cv['test_score'])


trials = Trials()
best = fmin(
            fn = kriteri_fn,
            space = gnezdeni_prostor,
            algo = tpe.suggest,
            max_evals = 100,
            trials = trials
            )

best = space_eval(gnezdeni_prostor, best)
best_value = kriteri_fn(best)




100%|██████████| 100/100 [00:22<00:00,  4.44trial/s, best loss: 0.06384257732235832]


In [59]:
print(f"Best algo with params {best}")
print(f"Best score {best_value}")

Best algo with params {'algo': {'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 5, 'name': 'rf'}}
Best score 0.0659539539319427


Sedaj pa se nataniram model z vsemi znacilkami in izracunanimi parametri

In [69]:

Xi_train = X_train[sort_name[0:10]]
Xi_test =  X_test[sort_name[0:10]]

np.random.seed(0)

bestmodel = RandomForestClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=5)
cv = cross_validate(bestmodel, Xi_train, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)

bestmodel.fit(Xi_train, y_train)
pred = bestmodel.predict(Xi_test)

print(f"AUC on cv = {np.mean(cv['test_score'])}")
print(f"AUC on test = {roc_auc_score(pred, y_test)}")



AUC on cv = 0.9347978383863558
AUC on test = 0.8298701298701299
